# 1. MQQ4 trials


In [1]:
%load_ext autoreload
%autoreload 2

import paho.mqtt.client as mqtt
from pydriller import RepositoryMining
import pandas as pd
import pathlib
import datetime
import plotly.graph_objects as go

In [2]:
list_received_messages = []

# The callback for when the client receives a CONNACK response from the server.
def on_connect(client, userdata, flags, rc):
    print("Connected with result code "+str(rc))

    # Subscribing in on_connect() means that if we lose the connection and
    # reconnect then subscriptions will be renewed.
    # client.subscribe("$SYS/#")

# The callback for when a PUBLISH message is received from the server.
def on_message(client, userdata, msg):
    list_received_messages.append(msg.topic+" "+str(msg.payload))


### Instantiate a client

In [3]:
# Client(client_id=””, clean_session=True, userdata=None, protocol=MQTTv311, transport=”tcp”)
client = mqtt.Client()

In [4]:
client.on_connect = on_connect
client.on_message = on_message

## Connect to a broker

http://www.steves-internet-guide.com/client-connections-python-mqtt/

In [5]:
# client.connect(host, port=1883, keepalive=60, bind_address="")

host = "192.168.1.23" # "mqtt.eclipse.org"
client.connect(host, 1883, 60)

0

In [6]:
client.loop_start() #start the loop. Activate handling of callbacks

Connected with result code 0


In [7]:
client.subscribe("house/light")
client.subscribe("house/toast")

(0, 2)

In [8]:
message_info = client.publish("house/light","gherthr")
message_info = client.publish("house/toast","45")

In [9]:
message_info.is_published()

True

In [10]:
list_received_messages

["house/light b'gherthr'", "house/toast b'45'"]

In [11]:
client.loop_stop() 

# 2. Pathlib trials

In [12]:
pathlib.Path("./")

PosixPath('.')

# 3. Formatting of data trials

In [13]:
caca = {'Registration Timestamp': [datetime.datetime(2020, 2, 1, 18, 52, 24, 684720)], 'Topic': ['house/light'], 'Value': [b'hrtht']}

In [14]:
pd.DataFrame.from_dict(caca)

,Registration Timestamp,Topic,Value
0,2020-02-01 18:52:24.684720,house/light,b'hrtht'


# 4. Pydrill for github

In [15]:
repo_dir = "https://github.com/manuwhs/dashboards.git"

In [16]:
repo_mining = RepositoryMining(repo_dir)

In [17]:
commits = repo_mining.traverse_commits()

In [19]:
for commit in commits:
    print('On {} the author {} includes {} modified files'.format(commit.committer_date, commit.author.name, len(commit.modifications)))
    for modification in commit.modifications:
        print('Filename {}'.format(modification.filename))
        if modification.filename[-3:] == ".py":
            # print('Diff: {}'.format(modification.diff))
            print(f"   Lines added: {modification.added}")
            print(f"   Lines removed: {modification.removed}")


for commit in commits:
    print('On {} the author {} includes {} modified files'.format(commit.committer_date, commit.author.name, len(commit.modifications)))
    for modification in commit.modifications:
        print('Filename {}'.format(modification.filename))
        if modification.filename[-3:] == ".py":
            # print('Diff: {}'.format(modification.diff))
            print(f"   Lines added: {modification.added}")
            print(f"   Lines removed: {modification.removed}")
            

On 2020-02-01 12:59:06+01:00 the author Manuel Montoya includes 1 modified files
Filename README.md
On 2020-02-01 13:58:58+01:00 the author Manuel Montoya includes 30 modified files
Filename Aptfile
Filename LICENSE
Filename app.json
Filename .gitignore
Filename Procfile
Filename README.md
Filename app.py
   Lines added: 710
   Lines removed: 0
Filename dash-logo.png
Filename resizing_script.js
Filename s1.css
Filename styles.css
Filename controls.py
   Lines added: 145
   Lines removed: 0
Filename Oil_and_Gas_Annual_Production__1985_-_2000.csv
Filename Oil_and_Gas_Annual_Production__Beginning_2001.csv
Filename OilandGasMetadata.html
Filename dash-logo copy.png
Filename points.pkl
Filename wellspublic.csv
Filename explore.ipynb
Filename requirements.txt
Filename animated1.gif
Filename screenshot1.png
Filename screenshot2.png
Filename screenshot3.png
Filename apps_directory_mapping.py
   Lines added: 1
   Lines removed: 0
Filename deploy
Filename init.R
Filename predeploy.py
   Lines ad

## Generate charts with the repo info

In [20]:
repo_dir = "https://github.com/manuwhs/traphing"
repo_mining = RepositoryMining(repo_dir)
commits = repo_mining.traverse_commits()

In [21]:
data = {"Commit Timestamp": [], "Author":[],
        "Added Python lines":[], "Removed Python lines":[], "Modified files": []}

for commit in commits:
    data["Author"].append(commit.author.name)
    data["Commit Timestamp"].append(commit.committer_date)
    data["Modified files"].append(len(commit.modifications))
    data["Added Python lines"].append(0)
    data["Removed Python lines"].append(0)
    
    for modification in commit.modifications:
        if modification.filename[-3:] == ".py":
            data["Added Python lines"][-1] += modification.added
            data["Removed Python lines"][-1] += modification.removed

df = pd.DataFrame.from_dict(data).set_index("Commit Timestamp").sort_index()

In [22]:
df

,Author,Added Python lines,Removed Python lines,Modified files
Commit Timestamp,,,,
2019-09-21 10:25:35+02:00,Manuel Montoya,0,0,1
2019-09-21 10:32:25+02:00,montoya,7498,0,167
2019-09-21 17:24:43+02:00,montoya,1825,819,200
2019-09-21 18:17:08+02:00,montoya,6128,1,79
2019-09-21 18:19:22+02:00,montoya,0,0,1
2019-09-21 18:20:40+02:00,montoya,0,6127,66
2019-09-22 12:23:08+02:00,montoya,4567,846,42
2019-09-22 22:16:42+02:00,montoya,522,186,14
2019-09-23 20:50:48+02:00,montoya,326,11,10


In [23]:
import datetime as dt
dates = [dt.datetime(year = x.year, month = x.month, day = x.day) for x in df.index]


fig = go.Figure()
fig.add_trace(go.Bar(x=dates,
                y= df["Added Python lines"],
                name='Added lines',
                marker_color='rgb(55, 83, 109)',
                # width = 10000000,
                ))
fig.add_trace(go.Bar(x=dates,
                y= df["Removed Python lines"],
                name='Removed lines',
                marker_color='rgb(26, 118, 255)'
                ))

fig.update_layout(
    title='Repo modifications',
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Lines added',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1 # gap between bars of the same location coordinate.
)

figure = go.FigureWidget(fig)
figure

FigureWidget({
    'data': [{'marker': {'color': 'rgb(55, 83, 109)'},
              'name': 'Added lines',
   …

In [26]:
import sys
sys.path.insert(0, "../")
from plotting import holidays

In [29]:
holidays(df)

FigureWidget({
    'data': [{'colorscale': [[False, '#eeeeee'], [True, '#76cf63']],
              'hoverinfo':…